<a href="https://colab.research.google.com/github/pallalavanya22/Fmml-labs/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [20]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %



1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
ANS.....:-
     The accuracy of the validation set can be impacted by changes in the percentage of the data allocated for validation. Here’s what typically happens:

 1. **Increasing the Percentage of the Validation Set:**
   - **More Representative Validation:** With more data in the validation set, the model’s performance metrics (like accuracy) become more reliable because they are evaluated on a broader sample that better represents the overall dataset.
   - **Potential Decrease in Training Performance:** However, increasing the validation set size reduces the amount of data available for training. This reduction in training data might hinder the model's learning, leading to poorer generalization and potentially lower validation accuracy.
   - **Stabilization of Validation Metrics:** With a larger validation set, the accuracy and other metrics are less likely to fluctuate, providing more consistent and reliable performance measures.

 2. **Reducing the Percentage of the Validation Set:**
   - **Less Representative Validation:** With fewer data points in the validation set, the evaluation may become less representative, leading to less reliable accuracy metrics. The smaller sample size can introduce more noise and variability, making the model’s performance appear better or worse than it actually is.
   - **Better Training Performance:** With more data available for training, the model may learn better and achieve higher performance on both the training and validation sets. However, this improved learning might not always translate to better generalization.
   - **Increased Variability in Validation Metrics:** A smaller validation set can lead to more fluctuation in accuracy, especially if it contains outliers or is not well-balanced.

 Conclusion:
Increasing the validation set size generally improves the reliability of the accuracy metric but may hurt model training due to less data being available for learning. Reducing the validation set size might enhance training but can lead to less reliable and more volatile validation metrics. A balanced approach is usually needed, often guided by cross-validation techniques to make the most of the available data.
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
ANS...:
The accuracy of the validation set can be impacted by changes in the percentage of data allocated for validation. Here’s how:

1. **Increasing the Percentage of the Validation Set:**
   - **More Reliable Accuracy Measurement:** A larger validation set provides a better representation of the overall data distribution. This makes the validation accuracy more stable and less prone to fluctuations, leading to a more reliable estimate of model performance.
   - **Less Training Data:** However, increasing the validation set reduces the amount of data available for training. With less training data, the model may not learn as effectively, potentially leading to poorer generalization and a decrease in validation accuracy.

 2. **Reducing the Percentage of the Validation Set:**
   - **Less Reliable Accuracy Measurement:** A smaller validation set means the accuracy may be more sensitive to the specific samples in that set. It could lead to less reliable and more variable accuracy measurements since a small subset may not fully represent the overall data distribution.
   - **More Training Data:** More data is available for training, which can help the model learn better. This can potentially improve both training and validation accuracy, though this improved performance might be due to overfitting if the validation set is too small.

 Summary:
- **Increasing the validation set size** generally provides a more reliable estimate of model accuracy but at the cost of reducing training data.
- **Decreasing the validation set size** can improve training by providing more data, but it risks making the validation accuracy less reliable and prone to variability.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
ANS...:
The size of the training and validation sets directly impacts how well the validation set can predict the model's performance on the test set. Here’s how:

1. **Training Set Size:**
   - **Larger Training Set:**
     - **Better Model Generalization:** A larger training set allows the model to learn more patterns from the data, improving its generalization ability. This leads to better performance on both the validation and test sets.
     - **More Accurate Validation Performance:** With more data to train on, the model can perform better on unseen data, so the validation accuracy is a better reflection of how well the model will perform on the test set.
   - **Smaller Training Set:**
     - **Limited Learning:** A smaller training set may lead to underfitting because the model might not learn enough from the limited data. As a result, validation accuracy may be poor, and it may not be a reliable predictor of test set performance.

 2. **Validation Set Size:**
   - **Larger Validation Set:**
     - **More Reliable Estimates of Test Accuracy:** A larger validation set provides a more representative sample of the data distribution, leading to more stable and accurate estimates of model performance. This improves the reliability of predicting how well the model will do on the test set.
     - **Less Training Data:** However, a larger validation set reduces the training data, which can hurt model performance if too little data is left for training. This could cause the validation accuracy to underestimate the true test accuracy.
   - **Smaller Validation Set:**
     - **Less Reliable Estimates of Test Accuracy:** A smaller validation set may not be representative of the overall data distribution. This can lead to high variance in validation accuracy, making it a less reliable indicator of test set performance.
     - **More Training Data:** With more data available for training, the model might learn better and achieve better performance, but this performance could be overestimated if the validation set is too small to capture the true variability of the data.

Summary:
- **Training Set Size:** A larger training set generally leads to better model generalization, allowing the validation set to more accurately predict test set performance.
- **Validation Set Size:** A larger validation set provides more reliable estimates of test accuracy but at the cost of reducing training data. A smaller validation set increases the risk of inaccurate or volatile predictions for test performance.

Balancing the size of the training and validation sets is crucial. Techniques like cross-validation can help mitigate the trade-offs by ensuring that all data is effectively used for both training and validation.
Here’s how you can implement a 3-Nearest Neighbors (KNN) classifier and compare it with the 1-Nearest Neighbors classifier using scikit-learn:

 Step 1: Import Necessary Libraries

```python
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
```

 Step 2: Implement and Compare the 1-Nearest and 3-Nearest Neighbors Classifiers

```python
# 1-Nearest Neighbor Classifier
knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_1.fit(alltraindata, alltrainlabel)
testpred_knn1 = knn_1.predict(testdata)
testAccuracy_knn1 = accuracy_score(testlabel, testpred_knn1)
print(f"Test accuracy using 1-Nearest Neighbor: {testAccuracy_knn1 * 100:.2f}%")

# 3-Nearest Neighbors Classifier
knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_3.fit(alltraindata, alltrainlabel)
testpred_knn3 = knn_3.predict(testdata)
testAccuracy_knn3 = accuracy_score(testlabel, testpred_knn3)
print(f"Test accuracy using 3-Nearest Neighbors: {testAccuracy_knn3 * 100:.2f}%")
```

 Step 3: Observe and Compare the Results
- The output will give you the accuracy of both the 1-Nearest Neighbor and 3-Nearest Neighbors classifiers on the test dataset.

 Explanation:
- **1-Nearest Neighbor Classifier**: This classifier looks for the closest training point to each test point and predicts the label of that closest point.
- **3-Nearest Neighbors Classifier**: This classifier looks for the 3 closest training points to each test point and predicts the majority label among those 3 neighbors.

Expected Outcome:
- The 3-Nearest Neighbors classifier typically gives better generalization compared to the 1-Nearest Neighbor classifier because it considers more neighbors, which reduces the impact of noise in the data.
- The accuracy might be slightly higher for the 3-Nearest Neighbors classifier, though it depends on the dataset.

Running the Code
You can run the above code in your environment to see the exact accuracy values for your specific dataset.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>


1. Does averaging the validation accuracy across multiple splits give more consistent results?
ANS...:
Yes, averaging the validation accuracy across multiple splits does generally provide more consistent and reliable results. This is a common technique in machine learning to reduce the variance of model performance estimates, making them more representative of the model's true generalization ability.

Why Does Averaging Improve Consistency?

1. **Reduces Variance**: When you train and validate your model on a single split, the accuracy can be overly dependent on that specific split. If that split happens to be particularly easy or difficult, the accuracy may be overly optimistic or pessimistic. By using multiple splits and averaging the results, you smooth out these inconsistencies.

2. **More Robust Estimation**: Different data splits can lead to different models, especially when the dataset is small. By averaging the results across several splits (like in k-fold cross-validation), you get a better sense of how your model is likely to perform on unseen data.

3. **Minimizes the Effect of Outliers**: In some splits, the validation set might contain outliers or uncommon data points that can disproportionately influence the model’s performance. By averaging across multiple splits, the effect of such outliers is diminished.

### Techniques That Use Averaging Across Splits:

1. **k-Fold Cross-Validation**: The dataset is split into `k` subsets. The model is trained `k` times, each time using `k-1` subsets for training and the remaining subset for validation. The final accuracy is the average accuracy across all `k` folds.

2. **Repeated k-Fold Cross-Validation**: To increase reliability, k-fold cross-validation can be repeated multiple times with different random splits. The final performance is then averaged across all the repetitions.

3. **Stratified k-Fold Cross-Validation**: If the dataset is imbalanced, stratified k-fold ensures each fold has roughly the same distribution of classes, leading to more consistent results.

### Example of Averaging with k-Fold Cross-Validation in Scikit-Learn:

```python
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

# Perform 5-fold cross-validation and average the results
cv_scores = cross_val_score(knn, alltraindata, alltrainlabel, cv=5)

# Average accuracy across all splits
average_accuracy = cv_scores.mean()
print(f"Average validation accuracy across 5 folds: {average_accuracy * 100:.2f}%")
```

### Conclusion:
Averaging validation accuracy across multiple splits (like in k-fold cross-validation) does indeed lead to more consistent and reliable results. It gives a better estimation of how well your model will generalize to unseen data and reduces the risk of overfitting or underfitting due to a particular split.
2. Does it give more accurate estimate of test accuracy?
ANS...:
Yes, averaging validation accuracy across multiple splits generally gives a more accurate estimate of test accuracy. Here's why:

### 1. **Generalization to Unseen Data**:
A single train-test split might produce a model that performs very well or very poorly due to the specific characteristics of that split. By using multiple splits (like in k-fold cross-validation), you assess your model's performance across different parts of the dataset. This gives a more comprehensive view of how the model might perform on unseen data.

### 2. **Reduces the Risk of Overfitting/Underfitting**:
When you rely on a single split, your model might overfit to that specific training set or underperform if the validation set is particularly challenging. Averaging across multiple splits smooths out these extremes, giving a more accurate picture of the model’s actual performance on unseen data.

### 3. **More Stable and Reliable Results**:
In practice, a model that consistently performs well across different splits is more likely to generalize better to new data. Since k-fold cross-validation evaluates the model on different subsets, the average validation accuracy provides a more stable estimate of the model’s true performance.

### 4. **Better Representation of the Dataset**:
In k-fold cross-validation, every data point gets a chance to be in the validation set exactly once. This reduces the bias in performance estimation since all data points are eventually used for both training and validation. As a result, the average validation accuracy is a closer approximation of how the model will perform on truly unseen test data.

### Real-World Application:
In many cases, models validated with k-fold cross-validation tend to have test accuracies that are closer to the averaged validation accuracies than models validated using a single train-test split. This is why k-fold cross-validation is often preferred in machine learning pipelines.

### Example of k-Fold Cross-Validation Estimating Test Accuracy:

```python
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

# Perform 5-fold cross-validation and average the results
cv_scores = cross_val_score(knn, alltraindata, alltrainlabel, cv=5)

# Average accuracy across all splits
estimated_test_accuracy = cv_scores.mean()
print(f"Estimated test accuracy using 5-fold cross-validation: {estimated_test_accuracy * 100:.2f}%")
```

### Summary:
Averaging validation accuracy across multiple splits gives a more accurate and realistic estimate of test accuracy than relying on a single train-test split. It better captures how the model will perform on new, unseen data by reducing the variance and bias inherent in individual splits.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
ANS..:
In iterative algorithms or processes, increasing the number of iterations generally improves the estimate's accuracy. Here's how:

1. **Convergence**: Many algorithms improve their estimates as they iterate, converging to a more accurate result. For example, in numerical methods, more iterations often lead to a more accurate solution.

2. **Error Reduction**: More iterations can help reduce errors by refining the estimate through repeated updates. This is especially true for algorithms that incrementally approach a solution.

3. **Diminishing Returns**: While more iterations usually improve accuracy, the improvements might diminish over time. After a certain point, additional iterations may offer only marginal gains.

4. **Computational Cost**: More iterations increase the computational cost. There’s a trade-off between accuracy and computational efficiency.

Overall, while more iterations tend to improve estimates, the actual benefit depends on the specific algorithm and problem context.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
ANS..:
Increasing the number of iterations can help improve the performance of a model, but it might not fully compensate for a very small training or validation dataset. Here’s why:

1. **Overfitting**: With a small dataset, increasing iterations might lead to overfitting, where the model learns the noise and specific details of the training data rather than generalizing well to new data.

2. **Limited Data Representation**: A small dataset might not provide enough variability or coverage of the feature space, making it hard for the model to learn meaningful patterns, regardless of how many iterations are used.

3. **Validation Performance**: If the validation dataset is very small, it might not reliably reflect the model’s performance. More iterations might help, but the validation results could still be noisy or unrepresentative.

4. **Regularization and Techniques**: To handle small datasets more effectively, consider using techniques like regularization, data augmentation, or cross-validation, which can help mitigate the issues associated with limited data.

In summary, while increasing iterations can be beneficial, addressing the issue of a small dataset often requires additional strategies beyond just more iterations.

When evaluating a k-nearest neighbors (k-NN) classifier, the accuracy can be influenced by the number of splits and split size, particularly in the context of cross-validation. Here's how these factors typically affect the performance of both 1-nearest neighbor (1-NN) and 3-nearest neighbor (3-NN) classifiers:

### Number of Splits

1. **More Splits (e.g., k-fold Cross-Validation)**:
   - **1-NN**: With more splits, the training set for each fold becomes smaller, which can increase variability in the performance metrics due to the smaller sample size used for training in each fold. However, because 1-NN is highly sensitive to noise and outliers, it may show higher variability in accuracy.
   - **3-NN**: The impact of more splits is generally less severe for 3-NN compared to 1-NN. The classifier averages the predictions of three nearest neighbors, which helps mitigate the effect of individual noisy or outlier data points. This can lead to more stable accuracy metrics as the number of splits increases.

2. **Fewer Splits**:
   - **1-NN**: With fewer splits, each fold has a larger training set, which can lead to more stable estimates of accuracy. However, if the dataset is small, a single large training set might still be prone to overfitting.
   - **3-NN**: Similar to 1-NN, fewer splits mean larger training sets per fold, which generally leads to more stable and less variable accuracy estimates. The 3-NN classifier's ability to generalize might improve compared to 1-NN due to reduced sensitivity to individual data points.

### Split Size

1. **Larger Split Size**:
   - **1-NN**: Larger training sets in each fold provide more data for the classifier, which can lead to more stable and potentially higher accuracy. However, the high sensitivity of 1-NN to individual instances means that large training sets might not fully resolve its issues with overfitting and noise.
   - **3-NN**: Larger training sets benefit 3-NN by providing a richer set of neighbors for classification, which generally leads to better generalization and more stable accuracy compared to 1-NN. The classifier is less affected by noise in larger datasets.

2. **Smaller Split Size**:
   - **1-NN**: Smaller training sets can lead to higher variance in accuracy and potential overfitting due to the classifier’s sensitivity to individual data points.
   - **3-NN**: Smaller training sets might still provide more stable accuracy compared to 1-NN, but the classifier’s performance can be more variable due to reduced neighborhood information.

### Summary

- **1-NN** is more sensitive to the number of splits and split size due to its reliance on a single nearest neighbor. It is likely to show higher variability in accuracy and might be more prone to overfitting with smaller splits.
- **3-NN** generally exhibits more stable accuracy with varying numbers of splits and split sizes, as it considers multiple neighbors for classification, reducing sensitivity to individual noisy data points.

In essence, 3-NN tends to be more robust and stable across different cross-validation setups compared to 1-NN.> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.